In [1]:
import numpy as np
from k_local import build_local_k

Q = np.array([
        [1,0],
        [0,1]
    ])

## Criação da discretização 2D, numeração dos nós e numeração dos elementos

In [2]:
nx = 7
ny = 5
X, Y = np.meshgrid(np.linspace(0,np.pi,nx), np.linspace(0,np.pi,ny))

In [3]:
Z = np.array(list(zip(X.ravel(), Y.ravel())))

In [4]:
nodes = list(enumerate(Z))
nodes_n = nodes[-1][0]

In [5]:
elements_n = (nx-1)*(ny-1)

In [6]:
G = np.zeros((4,elements_n))

In [7]:
num = 0
for d in range(ny-1):
    for i in range((nx-1)):
        G[:,num] = np.array([i+d*nx, i+1+d*nx, i+nx+1+d*nx, i+nx+d*nx])
        num+=1

## Montagem da matriz rigidez local, e posterior mapeamento para a matriz rigidez global

In [8]:
for x in range(G.shape[1]):
    n_ = G[:,x]
    l = list()
    for idx in n_:
        l.append(nodes[int(idx)][1])
    M = np.array(l).T
    K = build_local_k(M, Q)

In [9]:
def l2g(local_matrix, EQ_vector, L2G_matrix):
    #L2G_matrix sera o n_
    nodes_per_elem, elem_n = L2G_matrix.shape
    
    eq_n = EQ_vector[-1] + 1
    K = np.zeros((eq_n, eq_n))
    
    c = itertools.product(range(nodes_per_elem),repeat=2)
    for x in c:
        aux1 = L2G_matrix[x[0]]
        aux2 = L2G_matrix[x[1]]

        i = EQ_vector[aux1]
        j = EQ_vector[aux2]

        if i != -1 and j != -1:
            K[i][j] += local_matrix[x[0]][x[1]]

    return K